In [2]:
import matplotlib.pyplot as plt
import numpy as np
import random as rn
import cv2
from skimage import io,filters
from skimage.feature import graycomatrix, graycoprops
from skimage.color import rgb2lab, rgb2hsv, rgb2ycbcr #rgb2cmyk, rgb2hsi,
from skimage.io import imread
from skimage.util import img_as_ubyte
from Preprocessing_pipline import mask_scab, mask_rust_healthy
from skimage import color
from PIL import Image
import pandas as pd
from Feature_extraction_pipline import extract_color_features, features2array

In [14]:
df = pd.DataFrame(columns=['Image ID', 'Features Scab', 'Features Rust', 'Features Original', 'All Features'])
PATH= '..\\Base de datos\\BD 810\\'
for i in range (1,811,1): 
    org_image = cv2.imread(PATH + str(i) + ".jpg", cv2.IMREAD_COLOR)
    disease_segmented_1=mask_rust_healthy(org_image)
    disease_segmented_2=mask_scab(org_image)
    features_org = features2array(extract_color_features(org_image))
    features_rust = features2array(extract_color_features(disease_segmented_1))
    features_scab = features2array(extract_color_features(disease_segmented_2))
    new_line = ['image'+ str(i), features_scab, features_rust, features_org, features_scab + features_rust + features_org]
    # Agregar la nueva fila utilizando el método loc
    df.loc[len(df)] = new_line
    #new_line = {"Image ID":str(i), "Features Scab":features_scab, "Features Rust":features_rust, "Features Original":features_org}
    #df = df.append(new_line, ignore_index=True)
print (df)

df.to_csv('..\\Base de datos\\Feature_extraction.csv', index=False)

c:\Users\maria\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\histograms.py:883: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges
c:\Users\maria\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\histograms.py:883: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges
c:\Users\maria\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\histograms.py:883: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges
c:\Users\maria\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\histograms.py:883: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges
c:\Users\maria\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\histograms.py:883: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges
c:\Users\maria\AppData\Local\Programs\Python\Python311\Lib\site-p

     Image ID                                      Features Scab   
0      image1  [33.9112803141276, 58.328264497784645, 4.34704...  \
1      image2  [16.637369791666668, 34.52175187504406, 2.6702...   
2      image3  [29.535303751627605, 41.438059528524285, 3.478...   
3      image4  [21.416554768880207, 40.43419289746061, 3.2840...   
4      image5  [25.15814208984375, 51.20592483205036, 2.91167...   
..        ...                                                ...   
805  image806  [1.6905670166015625, 9.8086412040009, 0.219163...   
806  image807  [21.526743570963543, 39.193087669185154, 2.719...   
807  image808  [27.876541137695312, 47.683899311565085, 3.331...   
808  image809  [22.463602701822918, 30.097467869041093, 3.584...   
809  image810  [42.30254109700521, 58.886915390187326, 4.8935...   

                                         Features Rust   
0    [0.07916768391927083, 2.765539255195737, 0.006...  \
1    [0.0, 0.0, 0.0, 0.0, 255.0, 0.0, 0.41666666, 0...   
2    [0.0

In [15]:
df_complete = pd.read_csv('..\\Base de datos\\Feature_extraction.csv')
label = []
for i in range (1,811,1):
    if i <=270:
        label.append('Healthy')
    elif i<=540:
        label.append('Rust')
    else:
        label.append('Scab')

df_complete ['Label'] = label
df_complete.to_csv('..\\Base de datos\\Feature_extraction.csv', index=False)

In [21]:
def convertir_a_arreglo(string):
    arreglo = np.fromstring(string[1:-1], dtype=float, sep=',') #string[1:-1] eliminar primer y último carácter "[]"
    #print (string)
    return arreglo

df_complete['All Features'] = df_complete['All Features'].apply(convertir_a_arreglo)

def expandir_columnas(row):
    valores = row['All Features']
    for i, valor in enumerate(valores):
        row[f'Feature_{i+1}'] = valor
    return row

# Aplicar la función a cada fila del DataFrame
df_expandido = df_complete.apply(expandir_columnas, axis=1)
print (df_expandido)

     Image ID                                      Features Scab   
0      image1  [33.9112803141276, 58.328264497784645, 4.34704...  \
1      image2  [16.637369791666668, 34.52175187504406, 2.6702...   
2      image3  [29.535303751627605, 41.438059528524285, 3.478...   
3      image4  [21.416554768880207, 40.43419289746061, 3.2840...   
4      image5  [25.15814208984375, 51.20592483205036, 2.91167...   
..        ...                                                ...   
805  image806  [1.6905670166015625, 9.8086412040009, 0.219163...   
806  image807  [21.526743570963543, 39.193087669185154, 2.719...   
807  image808  [27.876541137695312, 47.683899311565085, 3.331...   
808  image809  [22.463602701822918, 30.097467869041093, 3.584...   
809  image810  [42.30254109700521, 58.886915390187326, 4.8935...   

                                         Features Rust   
0    [0.07916768391927083, 2.765539255195737, 0.006...  \
1    [0.0, 0.0, 0.0, 0.0, 255.0, 0.0, 0.41666666, 0...   
2    [0.0

In [23]:
df_expandido = df_expandido.drop (['Features Scab', 'Features Rust', 'Features Original', 'All Features'], axis = 1)
df_expandido.to_csv('..\\Base de datos\\Feature_extraction_extended.csv', index=False)
print (df_expandido)

     Image ID    Label  Feature_1  Feature_2  Feature_3  Feature_4   
0      image1  Healthy  33.911280  58.328264   4.347049  16.716625  \
1      image2  Healthy  16.637370  34.521752   2.670231  10.841078   
2      image3  Healthy  29.535304  41.438060   3.478244  14.052064   
3      image4  Healthy  21.416555  40.434193   3.284025  12.796891   
4      image5  Healthy  25.158142  51.205925   2.911672  15.131888   
..        ...      ...        ...        ...        ...        ...   
805  image806     Scab   1.690567   9.808641   0.219164   3.658032   
806  image807     Scab  21.526744  39.193088   2.719575  15.547844   
807  image808     Scab  27.876541  47.683899   3.331781  17.403050   
808  image809     Scab  22.463603  30.097468   3.584035  11.799607   
809  image810     Scab  42.302541  58.886915   4.893534  19.970152   

      Feature_5  Feature_6  Feature_7  Feature_8  ...  Feature_39  Feature_40   
0    221.088720  58.328264   0.377990   0.375360  ...    0.394782    0.293899 

In [18]:

# Crear el DataFrame a partir del diccionario
df_complete = pd.read_csv('..\\Base de datos\\Feature_extraction.csv')
columnas_nuevas = [f'All Features{i+1}' for i in range(len(df_complete['All Features'].values[0]))]
print (columnas_nuevas)
df_nuevo = pd.DataFrame(columnas_nuevas)

# for i in range (1,811,1):
#     columnas_nuevas = {'All Features{i+1}': df['All Features'] for i in range(len['All Features'])}

# df_complete ['Label'] = label
# df_complete.to_csv('..\\Base de datos\\Feature_extraction.csv', index=False)

['All Features1', 'All Features2', 'All Features3', 'All Features4', 'All Features5', 'All Features6', 'All Features7', 'All Features8', 'All Features9', 'All Features10', 'All Features11', 'All Features12', 'All Features13', 'All Features14', 'All Features15', 'All Features16', 'All Features17', 'All Features18', 'All Features19', 'All Features20', 'All Features21', 'All Features22', 'All Features23', 'All Features24', 'All Features25', 'All Features26', 'All Features27', 'All Features28', 'All Features29', 'All Features30', 'All Features31', 'All Features32', 'All Features33', 'All Features34', 'All Features35', 'All Features36', 'All Features37', 'All Features38', 'All Features39', 'All Features40', 'All Features41', 'All Features42', 'All Features43', 'All Features44', 'All Features45', 'All Features46', 'All Features47', 'All Features48', 'All Features49', 'All Features50', 'All Features51', 'All Features52', 'All Features53', 'All Features54', 'All Features55', 'All Features56', 